In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional

plt.style.use('fivethirtyeight')

In [2]:
#ger stock quote
df = web.DataReader('0011.HK',data_source='yahoo',start='2015-07-01',end='2022-01-31')
#show
print(df)


                  High         Low        Open       Close     Volume  \
Date                                                                    
2015-07-02  153.800003  150.500000  153.800003  151.699997  1031169.0   
2015-07-03  153.199997  151.300003  151.500000  151.800003   572293.0   
2015-07-06  152.800003  148.000000  152.800003  148.899994  1443473.0   
2015-07-07  151.000000  147.699997  150.000000  148.100006  1567244.0   
2015-07-08  148.000000  138.000000  148.000000  142.199997  5466707.0   
...                ...         ...         ...         ...        ...   
2022-01-25  157.500000  152.800003  154.800003  156.300003  1502848.0   
2022-01-26  157.699997  155.699997  156.300003  157.000000  1910629.0   
2022-01-27  157.699997  156.100006  157.600006  157.300003  1499879.0   
2022-01-28  158.300003  155.800003  158.000000  155.899994  2060189.0   
2022-01-31  153.699997  153.699997  153.699997  153.699997        0.0   

             Adj Close  
Date                    


In [3]:
#change % of close
ccp = (df['Close']-df['Close'].shift(1))/df['Close'].shift(1)
df["change % of close"] = ccp*100

In [4]:
# day average
ma5d = np.array([])
ma20d = np.array([])
for i in range(len(df['Close'])):
    if(i<4):
        ma5d = np.append(ma5d,np.nan)
    else:
        ma5d = np.append(ma5d,np.average(df['Close'][i-4:i+1]))
    if(i<19):
        ma20d = np.append(ma20d,np.nan)
    else:
        ma20d = np.append(ma20d,np.average(df['Close'][i-19:i+1]))
df['5-day moving average'] = ma5d
df['20-day moving average'] = ma20d
print(ma20d[-1])

154.37000045776367


In [5]:
#RSI
RSI_14 = np.array([])
RSI_21 = np.array([])
loss = np.array([])
gain = np.array([])
for i in range(len(df['Close'])):
    if i == 0:
        gain = np.append(gain,np.nan)
        loss = np.append(loss,np.nan)
    else:
        diff = df['Close'][i] - df['Close'][i-1]
        if diff > 0:
            gain = np.append(gain,diff)
            loss = np.append(loss,0)
        else:
            gain = np.append(gain,0)
            loss = np.append(loss,-diff)
for i in range(len(df['Close'])):
    if i < 14:
        RSI_14 = np.append(RSI_14,np.nan)
    else:
        RSI_14 = np.append(RSI_14,100-100/(1+np.average(gain[i-13:i+1])/np.average(loss[i-13:i+1])))
    if i < 21:
        RSI_21 = np.append(RSI_21,np.nan)
    else:
        RSI_21 = np.append(RSI_21,100-100/(1+np.average(gain[i-20:i+1])/np.average(loss[i-20:i+1])))
df['RSI 14'] = RSI_14

In [6]:
#MACD & Signal
# Get the 26-day EMA of the closing price
k = df['Close'].ewm(span=12, adjust=False, min_periods=12).mean()
# Get the 12-day EMA of the closing price
d = df['Close'].ewm(span=26, adjust=False, min_periods=26).mean()
# Subtract the 26-day EMA from the 12-Day EMA to get the MACD
macd = k - d
# Get the 9-Day EMA of the MACD for the Trigger line
Signal = macd.ewm(span=9, adjust=False, min_periods=9).mean()
# Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
Histogram = macd - Signal
df['MACD'] = macd
df['Signal'] = Signal
df['Histogram'] = Histogram

In [7]:
#Bollinger Bands
std = df['Close'].rolling(20).std()
bollinger_up = ma20d + std * 2 # Calculate top band
bollinger_down = ma20d - std * 2 # Calculate bottom band
df['Upper Bollinger Band'] = bollinger_up
df['Lower Bollinger Band'] = bollinger_down

In [8]:
#close next day
df["Close next day"] = df['Close'].shift(-1)

In [9]:
print(df)
df_train = df.query("index >= 20160101 & index <= 20201231")
df_train.to_excel("train.xlsx")
df_test = df.query("index >= 20210101 & index <= 20211231")
df_test.to_excel("test.xlsx")

                  High         Low        Open       Close     Volume  \
Date                                                                    
2015-07-02  153.800003  150.500000  153.800003  151.699997  1031169.0   
2015-07-03  153.199997  151.300003  151.500000  151.800003   572293.0   
2015-07-06  152.800003  148.000000  152.800003  148.899994  1443473.0   
2015-07-07  151.000000  147.699997  150.000000  148.100006  1567244.0   
2015-07-08  148.000000  138.000000  148.000000  142.199997  5466707.0   
...                ...         ...         ...         ...        ...   
2022-01-25  157.500000  152.800003  154.800003  156.300003  1502848.0   
2022-01-26  157.699997  155.699997  156.300003  157.000000  1910629.0   
2022-01-27  157.699997  156.100006  157.600006  157.300003  1499879.0   
2022-01-28  158.300003  155.800003  158.000000  155.899994  2060189.0   
2022-01-31  153.699997  153.699997  153.699997  153.699997        0.0   

             Adj Close  change % of close  5-day m